정리가 된 곳.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [454]:
import os

데이터 불러오기

In [156]:
RSB.columns

Index(['batter_id', 'batter_name', 'year', 'team', 'avg', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP',
       'SLG', 'OBP', 'E', 'height/weight', 'year_born', 'position', 'career',
       'starting_salary', 'OPS', 'lag_1_avg', 'lag_2_avg', 'lag_3_avg',
       'lag_1_G', 'lag_2_G', 'lag_3_G', 'lag_1_AB', 'lag_2_AB', 'lag_3_AB',
       'lag_1_R', 'lag_2_R', 'lag_3_R', 'lag_1_H', 'lag_2_H', 'lag_3_H',
       'lag_1_2B', 'lag_2_2B', 'lag_3_2B', 'lag_1_3B', 'lag_2_3B', 'lag_3_3B',
       'lag_1_HR', 'lag_2_HR', 'lag_3_HR', 'lag_1_TB', 'lag_2_TB', 'lag_3_TB',
       'lag_1_RBI', 'lag_2_RBI', 'lag_3_RBI', 'lag_1_SB', 'lag_2_SB',
       'lag_3_SB', 'lag_1_CS', 'lag_2_CS', 'lag_3_CS', 'lag_1_BB', 'lag_2_BB',
       'lag_3_BB', 'lag_1_HBP', 'lag_2_HBP', 'lag_3_HBP', 'lag_1_SO',
       'lag_2_SO', 'lag_3_SO', 'lag_1_GDP', 'lag_2_GDP', 'lag_3_GDP',
       'lag_1_SLG', 'lag_2_SLG', 'lag_3_SLG', 'lag_1_OBP', 'lag_2_OBP',
       'lag_3_OBP', 'lag_1_

In [159]:
PSB.columns

Index(['batter_id', 'batter_name', 'year', 'team', 'avg', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP',
       'SLG', 'OBP', 'E', 'height/weight', 'year_born', 'position', 'career',
       'starting_salary', 'OPS'],
      dtype='object')

In [160]:
RSB = pd.read_csv('RSB_lag.csv', delimiter=',', encoding='utf-8')
PSB = pd.read_csv('PSB_lag.csv', delimiter=',', encoding='utf-8')

DATA EDA

In [195]:
PSB = PSB[PSB['SLG'].notnull()]

In [219]:
PSB.loc[PSB['lag_1_avg']=='-','lag_1_avg']=np.nan

In [378]:
PSB.shape

(1364, 99)

모델 세우기

In [79]:
#train 지정하기.
lag_1list=[]
for i in ['avg', 'G', 'AB', 'R', 'H','2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E','1b_luck','2b_luck','3b_luck']:
    lag_1list.append('lag_1_'+i)
lag_2list=[]
for i in ['avg', 'G', 'AB', 'R', 'H','2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E','1b_luck','2b_luck','3b_luck']:
    lag_2list.append('lag_2_'+i)
lag_3list=[]
for i in ['avg', 'G', 'AB', 'R', 'H','2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E','1b_luck','2b_luck','3b_luck']:
    lag_3list.append('lag_3_'+i)

def make_train_test(Data,year,lag=lag_1list): #2018년을 넣으면, lag에 의해 2017년 데이터로 측정이 시작된다.
    train_set, test_set = train_test_split(Data.loc[(Data['year']==year)&(Data['OBP'].notnull())&(RSB['lag_1_OBP'].notnull())], test_size = 0.20)
    x_data = train_set[train_set['year']==year][lag]
    y_data = train_set[train_set['year']==year]['OPS']
    x_test = test_set[test_set['year']==year][lag]
    y_test = test_set[test_set['year']==year]['OPS']
    y_data=np.array(y_data).astype('float32').reshape(len(y_data),1)
    y_test=np.array(y_test).astype('float32').reshape(len(y_test),1)
    return x_data, y_data, x_test, y_test

def make_train_test_all(Data,lag=lag_1list): #모든 데이터셋
    train_set, test_set = train_test_split(Data.loc[(Data['OBP'].notnull())&(RSB['lag_1_OBP'].notnull())], test_size = 0.25)
    x_data = train_set[lag]
    y_data_OBP = train_set['OBP']
    y_data_SLG = train_set['SLG']
    y_OPS_data=train_set['OPS']
    
    x_test = test_set[lag]
    y_test_OBP = test_set['OBP']
    y_test_SLG = test_set['SLG']
    y_OPS_test=test_set['OPS']
    
    y_data_OBP=np.array(y_data_OBP).astype('float32').reshape(len(y_data_OBP),1)
    y_data_SLG=np.array(y_data_SLG).astype('float32').reshape(len(y_data_SLG),1)
    y_OPS_data=np.array(y_OPS_data).astype('float32').reshape(len(y_OPS_data),1)
    
    y_test_OBP=np.array(y_test_OBP).astype('float32').reshape(len(y_test_OBP),1)
    y_test_SLG=np.array(y_test_SLG).astype('float32').reshape(len(y_test_SLG),1)
    y_OPS_test=np.array(y_OPS_test).astype('float32').reshape(len(y_OPS_test),1)
    
    return x_data, y_data_OBP, y_data_SLG, y_OPS_data, x_test, y_test_OBP, y_test_SLG, y_OPS_test


x_data1, y_data1, x_test1, y_test1 = make_train_test(RSB,2018,lag_1list)

In [4]:
use_att=len(['avg', 'G', 'AB', 'R', 'H','2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E','1b_luck','2b_luck','3b_luck'])

X = tf.placeholder(tf.float32, [None, use_att])
Y = tf.placeholder(tf.float32, [None, 1])

# L1
W1 = tf.Variable(tf.random_normal([use_att,32], stddev=0.01))
b1 = tf.Variable(tf.random_normal([32]))
L1 = tf.nn.relu(tf.matmul(X,W1)+b1)

# L2 
W2 = tf.Variable(tf.random_normal([32, 32], stddev=0.01))
b2= tf.Variable(tf.random_normal([32]))
L2 = tf.nn.relu(tf.matmul(L1,W2)+b2)

#L2 = tf.reshape(L2,shape=[32,])
# L3

W3 = tf.get_variable('W3',shape=[32,1],
                     initializer=tf.contrib.layers.xavier_initializer())
pred = tf.matmul(L2, W3)

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Hyperparameter

In [138]:
learning_rate = 0.005

In [28]:
x_data1.fillna(0).isnull().sum()

lag_1_avg        0
lag_1_G          0
lag_1_AB         0
lag_1_R          0
lag_1_H          0
lag_1_2B         0
lag_1_3B         0
lag_1_HR         0
lag_1_TB         0
lag_1_RBI        0
lag_1_SB         0
lag_1_CS         0
lag_1_BB         0
lag_1_HBP        0
lag_1_SO         0
lag_1_GDP        0
lag_1_SLG        0
lag_1_OBP        0
lag_1_E          0
lag_1_1b_luck    0
lag_1_2b_luck    0
lag_1_3b_luck    0
dtype: int64

In [98]:
tf.square(Y-pred)

<tf.Tensor 'Square_21:0' shape=(?, 1) dtype=float32>

In [33]:
def sess_maker(Data,label,loop=5000):
    # define cost/loss & optimizer
    tocost=np.array(Data['lag_1_AB']).astype('float32').reshape(1,len(Data['lag_1_AB']))
    cost = tf.sqrt(tf.reduce_sum(tf.matmul(tocost,tf.square(Y-pred))))/tf.reduce_sum(Data['lag_1_AB'].astype('float32'))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    # train my model
    print('Learning started. It takes sometime.')
    for step in range(loop):
        sess.run(optimizer,feed_dict={X:Data, Y:label})
        if step %250 ==0:
            print(step, sess.run(cost,feed_dict={X:Data, Y:label}))

    print('Learning Finished!')
    return sess

In [38]:
sess1=sess_maker(x_data1.fillna(0),y_data1)

Learning started. It takes sometime.
0 0.0102824075
250 0.00061643805
500 0.0005152383
750 0.00043824318
1000 0.00062893337
1250 0.0006055768
1500 0.00062449946
1750 0.0005872362
2000 0.00069714524
2250 0.0006291245
2500 0.0006624172
2750 0.00057281496
3000 0.0006288512
3250 0.0006571377
3500 0.00070358213
3750 0.00065128313
4000 0.00063591124
4250 0.0006377074
4500 0.0006269961
4750 0.0006423306
Learning Finished!


In [35]:
#평가
def check_acc(data,label,lag, sess):
    ab= data[lag]
    return mean_squared_error(label,sess.run(pred,feed_dict={X:data, Y:label}),sample_weight=ab)**0.5

In [39]:
check_acc(x_test1,y_test1,'lag_1_AB',sess1)

0.13883143626300987

In [81]:
x_test.isnull().sum()

lag_1_avg        0
lag_1_G          0
lag_1_AB         0
lag_1_R          0
lag_1_H          0
lag_1_2B         0
lag_1_3B         0
lag_1_HR         0
lag_1_TB         0
lag_1_RBI        0
lag_1_SB         0
lag_1_CS         0
lag_1_BB         0
lag_1_HBP        0
lag_1_SO         0
lag_1_GDP        0
lag_1_SLG        0
lag_1_OBP        0
lag_1_E          0
lag_1_1b_luck    3
lag_1_2b_luck    3
lag_1_3b_luck    3
dtype: int64

In [80]:
x_data, y_data_OBP, y_data_SLG, y_OPS_data, x_test, y_test_OBP, y_test_SLG, y_OPS_test = make_train_test_all(RSB,lag_1list)

In [67]:
sess_all=sess_maker(x_data.fillna(0),y_data)

Learning started. It takes sometime.
0 0.0017213629
250 0.00022740995
500 0.00022490045
750 0.00022248476
1000 0.00022017177
1250 0.00021729393
1500 0.00021672838
1750 0.00021392202
2000 0.00021417822
2250 0.00020938944
2500 0.00020750125
2750 0.00020544535
3000 0.00020394879
3250 0.00020289388
3500 0.00020112976
3750 0.00019997227
4000 0.00020078362
4250 0.00019829602
4500 0.00019602507
4750 0.00019381638
Learning Finished!


In [68]:
check_acc(x_test.fillna(0),y_test,'lag_1_AB',sess_all)

0.15003392234430024

In [69]:
len(sess_all.run(pred,feed_dict={X:x_data, Y:y_data}))

1405

In [441]:
use_att=len(['avg', 'G', 'AB', 'R', 'H','2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E','1b_luck','2b_luck','3b_luck'])

X = tf.placeholder(tf.float32, [None, use_att])
Y1 = tf.placeholder(tf.float32, [None, 1])

# L1
W1 = tf.Variable(tf.random_normal([use_att,32], stddev=0.01))
b1 = tf.Variable(tf.random_normal([32]))
L1 = tf.nn.relu(tf.matmul(X,W1)+b1)

# L2 
W2 = tf.Variable(tf.random_normal([32, 32], stddev=0.01))
b2= tf.Variable(tf.random_normal([32]))
L2 = tf.nn.relu(tf.matmul(L1,W2)+b2)

#L2 = tf.reshape(L2,shape=[32,])
# L3

W3 = tf.get_variable('W_OBP_3',shape=[32,1],
                     initializer=tf.contrib.layers.xavier_initializer())

pred_OBP = tf.matmul(L2, W3)

Y2 = tf.placeholder(tf.float32, [None, 1])

# L1
W1_1 = tf.Variable(tf.random_normal([use_att,32], stddev=0.01))
b1_1 = tf.Variable(tf.random_normal([32]))
L1_1 = tf.nn.relu(tf.matmul(X,W1_1)+b1_1)

# L2 
W2_1 = tf.Variable(tf.random_normal([32, 32], stddev=0.01))
b2_1= tf.Variable(tf.random_normal([32]))
L2_1 = tf.nn.relu(tf.matmul(L1_1,W2_1)+b2_1)

#L2 = tf.reshape(L2,shape=[32,])
# L3

W3_3 = tf.get_variable('W_SLG_3',shape=[32,1],
                     initializer=tf.contrib.layers.xavier_initializer())

pred_SLG = tf.matmul(L2_1,W3_3)

Y3 = tf.placeholder(tf.float32, [None, 1])
pred = pred_SLG + pred_OBP

In [125]:
def sess_maker2(Data,y_test1,y_test2,label,loop=5000):
    # define cost/loss & optimizer
    tocost=np.array(Data['lag_1_AB']).astype('float32').reshape(1,len(Data['lag_1_AB']))
    cost = tf.sqrt(tf.reduce_sum(tf.matmul(tocost,tf.square(Y3-pred))))/tf.reduce_sum(Data['lag_1_AB'].astype('float32')) +tf.reduce_sum(
            tf.square(pred_SLG-Y1)) +tf.reduce_sum(tf.square(pred_OBP-Y2)) 
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    # train my model
    print('Learning started. It takes sometime.')
    for step in range(loop):
        sess.run(optimizer,feed_dict={X:Data, Y1:y_test1, Y2:y_test2, Y3:label})
        if step %250 ==0:
            print(step, sess.run(cost,feed_dict={X:Data, Y1:y_test1, Y2:y_test2, Y3:label}))

    print('Learning Finished!')
    return sess

In [376]:
tocost=np.array(x_data['lag_1_AB']).astype('float32').reshape(1,len(x_data['lag_1_AB']))
cost = tf.sqrt(tf.reduce_sum(tf.matmul(tocost,tf.square(Y3-pred))))/tf.reduce_sum(x_data['lag_1_AB'].astype('float32')) +tf.reduce_sum(
            tf.square(pred_SLG-Y1)) +tf.reduce_sum(tf.square(pred_OBP-Y2)) 
    
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
sess_all = tf.Session()
sess_all.run(tf.global_variables_initializer())
    
    
    # train my model
print('Learning started. It takes sometime.')
for step in range(5000):
    sess_all.run(optimizer,feed_dict={X:x_data.fillna(0.5), Y1:y_data_OBP, Y2:y_data_SLG, Y3:y_OPS_data})
    if step %250 ==0:
        print(step, sess_all.run(cost,feed_dict={X:x_data.fillna(0.5), Y1:y_data_OBP, Y2:y_data_SLG, Y3:y_OPS_data}))

print('Learning Finished!')

Learning started. It takes sometime.
0 4624.034
250 29.681599
500 24.34504
750 23.99105
1000 23.709328
1250 22.698166
1500 22.48937
1750 21.909796
2000 21.778528
2250 21.506325
2500 21.48972
2750 20.985939
3000 20.725428
3250 20.87026
3500 19.977608
3750 19.670692
4000 19.671772
4250 20.120968
4500 19.552704
4750 18.63498
Learning Finished!


In [320]:
len(x_data)

1405

In [312]:
sess_all=sess_maker2(x_data.fillna(0.5), y_data_OBP, y_data_SLG, y_OPS_data, 10000)

Learning started. It takes sometime.
0 1003.3911
250 25.526884
500 24.903358
750 24.359055
1000 24.07186
1250 23.540215
1500 22.86173
1750 21.863728
2000 21.307365
2250 20.869165
2500 20.115307
2750 20.097248
3000 19.33234
3250 19.296291
3500 18.449635
3750 18.049896
4000 17.830238
4250 17.83266
4500 22.21408
4750 21.880753
5000 21.530308
5250 21.126062
5500 21.260506
5750 22.186245
6000 21.159872
6250 21.023863
6500 20.408625
6750 20.563292
7000 20.23209
7250 21.162994
7500 20.114132
7750 20.181686
8000 19.927559
8250 20.58832
8500 20.278994
8750 19.92818
9000 20.02182
9250 20.055641
9500 20.303982
9750 19.930725
Learning Finished!


In [121]:
def check_acc2(Data,y_test1,y_test2,label,lag, sess):
    ab= Data[lag]
    return mean_squared_error(label,sess.run(pred,feed_dict={X:Data, Y1:y_test1, Y2:y_test2, Y3:label}),sample_weight=ab)**0.5

In [453]:
check_acc2(x_test.fillna(0.5), y_test_OBP, y_test_SLG, y_OPS_test,'lag_1_AB',sess_all)

0.13843929791780898

In [130]:
## PSB의 파일을 가지고 추가로 훈련 시킨 다음에, 입력한다.

lag_1_avg         0
lag_1_G           0
lag_1_AB          0
lag_1_R           0
lag_1_H           0
lag_1_2B          0
lag_1_3B          0
lag_1_HR          0
lag_1_TB          0
lag_1_RBI         0
lag_1_SB          0
lag_1_CS          0
lag_1_BB          0
lag_1_HBP         0
lag_1_SO          0
lag_1_GDP         0
lag_1_SLG         0
lag_1_OBP         0
lag_1_E           0
lag_1_1b_luck    10
lag_1_2b_luck    10
lag_1_3b_luck    10
dtype: int64

In [187]:
PSB.columns

Index(['batter_id', 'batter_name', 'year', 'team', 'avg', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP',
       'SLG', 'OBP', 'E', 'height/weight', 'year_born', 'position', 'career',
       'starting_salary', 'OPS', '1B', '1b_luck', '2b_luck', '3b_luck',
       'lag_1_avg', 'lag_2_avg', 'lag_3_avg', 'lag_1_G', 'lag_2_G', 'lag_3_G',
       'lag_1_AB', 'lag_2_AB', 'lag_3_AB', 'lag_1_R', 'lag_2_R', 'lag_3_R',
       'lag_1_H', 'lag_2_H', 'lag_3_H', 'lag_1_2B', 'lag_2_2B', 'lag_3_2B',
       'lag_1_3B', 'lag_2_3B', 'lag_3_3B', 'lag_1_HR', 'lag_2_HR', 'lag_3_HR',
       'lag_1_TB', 'lag_2_TB', 'lag_3_TB', 'lag_1_RBI', 'lag_2_RBI',
       'lag_3_RBI', 'lag_1_SB', 'lag_2_SB', 'lag_3_SB', 'lag_1_CS', 'lag_2_CS',
       'lag_3_CS', 'lag_1_BB', 'lag_2_BB', 'lag_3_BB', 'lag_1_HBP',
       'lag_2_HBP', 'lag_3_HBP', 'lag_1_SO', 'lag_2_SO', 'lag_3_SO',
       'lag_1_GDP', 'lag_2_GDP', 'lag_3_GDP', 'lag_1_SLG', 'lag_2_SLG',
       'lag_3_SLG', 'lag_1_

In [193]:
PSB[['batter_id', 'batter_name', 'year', 'team', 'avg', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP',
       'SLG', 'OBP', 'E']][PSB['SLG'].notnull()].isnull().sum()

batter_id      0
batter_name    0
year           0
team           0
avg            0
G              0
AB             0
R              0
H              0
2B             0
3B             0
HR             0
TB             0
RBI            0
SB             0
CS             0
BB             0
HBP            0
SO             0
GDP            0
SLG            0
OBP            0
E              0
dtype: int64

<bound method Tensor.eval of <tf.Tensor 'add_44:0' shape=() dtype=float32>>

In [364]:
test_PSB.isnull().sum()

lag_1_avg        0
lag_1_G          0
lag_1_AB         0
lag_1_R          0
lag_1_H          0
lag_1_2B         0
lag_1_3B         0
lag_1_HR         0
lag_1_TB         0
lag_1_RBI        0
lag_1_SB         0
lag_1_CS         0
lag_1_BB         0
lag_1_HBP        0
lag_1_SO         0
lag_1_GDP        0
lag_1_SLG        0
lag_1_OBP        0
lag_1_E          0
lag_1_1b_luck    0
lag_1_2b_luck    0
lag_1_3b_luck    0
dtype: int64

In [374]:
tocost=np.array(PSB['lag_1_AB'].fillna(0)).astype('float32').reshape(1,len(PSB['lag_1_AB']))
cost = tf.sqrt(tf.reduce_sum(tf.matmul(tocost,tf.square(Y3-pred))))/tf.reduce_sum(PSB['lag_1_AB'].fillna(0).astype('float32')) +tf.reduce_sum(
            tf.square(pred_SLG-Y1)) +tf.reduce_sum(tf.square(pred_OBP-Y2)) 

PSB_data_OBP=np.array(PSB['OBP']).astype('float32').reshape(len(PSB['OBP']),1)
PSB_data_SLG=np.array(PSB['SLG']).astype('float32').reshape(len(PSB['SLG']),1)
PSB_OPS_data=np.array(PSB['OPS']).astype('float32').reshape(len(PSB['OPS']),1)
test_PSB=PSB[lag_1list].fillna(0.5).astype('float32')

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate*0.0001).minimize(cost)
    
print("Learning Start")
for step in range(2500):
        sess_all.run(optimizer,feed_dict={X:test_PSB, Y1:PSB_data_OBP, Y2:PSB_data_SLG, Y3:PSB_OPS_data})
        if step %250 ==0:
            print(step, sess_all.run(cost,feed_dict={X:test_PSB, Y1:PSB_data_OBP, Y2:PSB_data_SLG, Y3:PSB_OPS_data}))
print("Learning Finish")

Learning Start
0 131.40402
250 130.50339
500 129.60663
750 128.76767
1000 127.97056
1250 127.251175
1500 126.61537
1750 125.977234
2000 125.34732
2250 124.73525
Learning Finish


In [389]:
everydata = pd.concat([test_PSB,x_data])
everyOBP= np.vstack([PSB_data_OBP,y_data_OBP])
everySLG=np.vstack([PSB_data_SLG,y_data_SLG])
everyOPS=np.vstack([PSB_OPS_data,y_OPS_data])
everydata.shape

(2769, 22)

In [446]:
tocost.shape

(1, 2769)

In [410]:
everyOBP.shape

(2769, 1)

In [388]:
np.vstack([PSB_data_OBP,y_data_OBP]).shape

(2769, 1)

In [452]:
tocost=np.array(everydata['lag_1_AB']).astype('float32').reshape(1,len(everydata['lag_1_AB']))
cost = tf.sqrt( tf.reduce_sum(tf.matmul(tocost,tf.square(Y3-pred))) /
               tf.reduce_sum(everydata['lag_1_AB'].astype('float32'))) + tf.reduce_sum(
            tf.square(pred_SLG-Y1)) + tf.reduce_sum(
    tf.square(pred_OBP-Y2)) 
        
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
sess_all = tf.Session()
sess_all.run(tf.global_variables_initializer())
    
    
    # train my model
print('Learning started. It takes sometime.')
for step in range(5000):
    sess_all.run(optimizer,feed_dict={X:everydata.fillna(0.5), Y1:everyOBP, Y2:everySLG, Y3:everyOPS})
    if step %250 ==0:
        print(step, sess_all.run(cost,feed_dict={X:everydata.fillna(0.5), Y1:everyOBP, Y2:everySLG, Y3:everyOPS}))

print('Learning Finished!')

Learning started. It takes sometime.
0 3517.1943
250 150.22156
500 148.31107
750 146.4072
1000 143.6871
1250 140.64145
1500 134.69025
1750 129.07257
2000 126.12631
2250 124.60428
2500 122.77716
2750 121.050766
3000 118.13347
3250 118.34652
3500 117.70659
3750 114.48526
4000 113.79787
4250 113.96625
4500 111.66997
4750 110.88977
Learning Finished!


In [451]:
print(sess_all.run(Y3,feed_dict={X:everydata.fillna(0.5), Y1:everyOBP, Y2:everySLG, Y3:y_OPS_data}).shape)

(1405, 1)


In [223]:
PSB['OBP'].dtype

dtype('float64')

In [455]:
SAVER_DIR = "model"
saver = tf.train.Saver()
checkpoint_path = os.path.join(SAVER_DIR, "model")
ckpt = tf.train.get_checkpoint_state(SAVER_DIR)

In [457]:
saver.save(sess_all, checkpoint_path)

'model\\model'

submission용 데이터 제작.

In [458]:
submission = pd.read_csv('submission.csv', delimiter=',', encoding='utf-8')

In [584]:
RSB[(RSB['year']==np.max(RSB['year'][RSB['batter_id']==0])) & (RSB['batter_id']==0)].head()

,batter_id,batter_name,year,team,avg,G,AB,R,H,2B,...,3b_luck,lag_1_1b_luck,lag_2_1b_luck,lag_3_1b_luck,lag_1_2b_luck,lag_2_2b_luck,lag_3_2b_luck,lag_1_3b_luck,lag_2_3b_luck,lag_3_3b_luck
0,0,가르시아,2018,LG,0.339,50,183,27,62,9,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [557]:
RSB[RSB['year']==np.max(RSB['year'][RSB['batter_id']==1]) & RSB['batter_id']==1]

ValueError: can only convert an array of size 1 to a Python scalar

In [593]:
latedata = RSB[:0]
for id  in RSB['batter_id'].unique():
    latedata = latedata.append(RSB[(RSB['year']==np.max(RSB['year'][RSB['batter_id']==id])) & (RSB['batter_id']==id)])

In [594]:
latedata.head()

,batter_id,batter_name,year,team,avg,G,AB,R,H,2B,...,3b_luck,lag_1_1b_luck,lag_2_1b_luck,lag_3_1b_luck,lag_1_2b_luck,lag_2_2b_luck,lag_3_2b_luck,lag_1_3b_luck,lag_2_3b_luck,lag_3_3b_luck
0,0,가르시아,2018,LG,0.339,50,183,27,62,9,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,강경학,2018,한화,0.278,77,245,42,68,11,...,0.005525,0.230769,0.142857,0.266932,0.030769,0.042857,0.027888,0.015385,0.028571,0.015936
11,2,강구성,2018,NC,0.200,4,5,0,1,1,...,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.200000,NaN,0.000000,0.000000
18,138,백승룡,2013,한화,NaN,2,0,0,0,0,...,NaN,0.343750,0.184211,NaN,0.031250,0.052632,NaN,0.000000,0.000000,NaN
37,297,진갑용,2015,삼성,0.324,39,68,8,22,2,...,0.000000,0.437500,0.231293,0.283525,0.000000,0.061224,0.057471,0.000000,0.000000,0.003831


In [595]:
latedata_true = RSB[:0]
for id in submission['batter_id'].unique():
    latedata_true = latedata_true.append(latedata[latedata['batter_id']==id])

In [592]:
latedata['batter_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [606]:
latedata_true

,batter_id,batter_name,year,team,avg,G,AB,R,H,2B,...,3b_luck,lag_1_1b_luck,lag_2_1b_luck,lag_3_1b_luck,lag_1_2b_luck,lag_2_2b_luck,lag_3_2b_luck,lag_1_3b_luck,lag_2_3b_luck,lag_3_3b_luck
6,1,강경학,2018,한화,0.278,77,245,42,68,11,...,0.005525,0.230769,0.142857,0.266932,0.030769,0.042857,0.027888,0.015385,0.028571,0.015936
11,2,강구성,2018,NC,0.200,4,5,0,1,1,...,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.200000,NaN,0.000000,0.000000
41,3,강민국,2018,NC,0.250,6,4,0,1,0,...,0.000000,NaN,0.076923,0.000000,NaN,0.076923,0.000000,NaN,0.000000,0.000000
56,4,강민호,2018,삼성,0.269,129,427,46,115,14,...,0.000000,0.257576,0.285185,0.274678,0.066667,0.096296,0.077253,0.003030,0.000000,0.004292
57,5,강백호,2018,KT,0.290,138,527,108,153,32,...,0.005348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,8,강상원,2018,한화,0.000,6,6,0,0,0,...,0.000000,0.222222,NaN,NaN,0.111111,NaN,NaN,0.000000,NaN,NaN
88,9,강승호,2018,SK,0.255,69,184,14,47,10,...,0.000000,0.232955,0.153846,NaN,0.079545,0.115385,NaN,0.011364,0.000000,NaN
100,11,강진성,2018,NC,0.235,45,68,5,16,3,...,0.000000,0.181818,NaN,NaN,0.136364,NaN,NaN,0.000000,NaN,NaN
105,12,강한울,2018,삼성,0.259,83,193,20,50,5,...,0.012579,0.318310,0.257937,0.201878,0.025352,0.051587,0.037559,0.008451,0.011905,0.014085
119,16,고명성,2018,KT,0.000,5,2,0,0,0,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


22

In [605]:
print(sess_all.run(pred,feed_dict={X:latedata_true[['avg', 'G', 'AB', 'R', 'H','2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E','1b_luck','2b_luck','3b_luck']].fillna(0.5), 
                                   Y1: np.array(latedata_true['OBP']).reshape(len(submission),1),
                                   Y2: np.array(latedata_true['SLG']).reshape(len(submission),1),
                                   Y3 :np.array(latedata_true['OPS']).reshape(len(submission),1) }))

[[ 0.7476582 ]
 [ 0.7963965 ]
 [ 0.6528629 ]
 [ 0.7978344 ]
 [ 0.898454  ]
 [ 0.80100703]
 [ 0.64660215]
 [ 0.46403676]
 [ 0.57648987]
 [ 0.4858042 ]
 [ 0.70155907]
 [ 0.8676231 ]
 [ 0.54231274]
 [ 0.7462746 ]
 [ 0.76343787]
 [ 0.81174785]
 [ 0.6835366 ]
 [ 0.6803742 ]
 [ 0.8202846 ]
 [ 0.59126824]
 [ 0.6797222 ]
 [ 0.70913464]
 [ 0.5438067 ]
 [ 0.6924627 ]
 [ 0.38223016]
 [ 0.8031029 ]
 [ 0.5819783 ]
 [ 0.74458504]
 [ 0.750468  ]
 [ 0.75212395]
 [ 0.7267506 ]
 [ 0.6638698 ]
 [ 0.643715  ]
 [ 0.5986921 ]
 [ 0.6353006 ]
 [ 0.5483124 ]
 [ 0.6290486 ]
 [ 0.83434665]
 [ 1.0210042 ]
 [ 0.61699855]
 [ 0.87400365]
 [ 0.7516263 ]
 [ 0.1813938 ]
 [ 0.52970886]
 [ 0.67554015]
 [ 0.72916543]
 [ 0.5283054 ]
 [ 0.56652653]
 [ 0.7348742 ]
 [ 0.8929931 ]
 [ 0.80056   ]
 [ 0.9371508 ]
 [ 0.42477763]
 [ 0.665931  ]
 [ 0.71613234]
 [ 0.68972695]
 [ 0.8715856 ]
 [ 0.64183676]
 [ 0.6342461 ]
 [ 0.7856306 ]
 [ 0.8963191 ]
 [ 0.5833888 ]
 [ 0.75619394]
 [ 0.724614  ]
 [ 0.7146879 ]
 [ 0.9995642 ]
 [ 1.00471

In [608]:
answer = sess_all.run(pred,feed_dict={X:latedata_true[['avg', 'G', 'AB', 'R', 'H','2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E','1b_luck','2b_luck','3b_luck']].fillna(0.5), 
                                   Y1: np.array(latedata_true['OBP']).reshape(len(submission),1),
                                   Y2: np.array(latedata_true['SLG']).reshape(len(submission),1),
                                   Y3 :np.array(latedata_true['OPS']).reshape(len(submission),1) })

In [609]:
submission['batter_ops']=answer

In [610]:
submission.head()

,batter_id,batter_name,batter_ops
0,1,강경학,0.747658
1,2,강구성,0.796396
2,3,강민국,0.652863
3,4,강민호,0.797834
4,5,강백호,0.898454


In [611]:
#submission.to_csv(r'C:\Users\74857\데이콘야구\submission_answer.csv',header=True,index=False)

In [440]:
#tf.reset_default_graph()